# Exploratory Data Analysis

> It is important to understand what you CAN DO before you learn to measure how WELL you seem to have DONE it.  

> Examples, NOT case histories

> The greatest value of a picture is when it forces us to notice what we never expected to see.

> To learn about data analysis, it is right that each of us try many things that do not work-that we tackle more problems than we make expert analyses of.

John W. Tukey, Exploratory Data Analysis (1977)



<img src='./diagrams/eda-tukey.jpg'>

> In statistics, exploratory data analysis is an approach of analyzing data sets to summarize their main characteristics, often using statistical graphics and other data visualization methods. A statistical model can be used or not, but primarily EDA is for seeing what the data can tell us beyond the formal modeling or hypothesis testing task. Exploratory data analysis was promoted by John Tukey to encourage statisticians to explore the data, and possibly formulate hypotheses that could lead to new data collection and experiments. EDA is different from initial data analysis (IDA),[1] which focuses more narrowly on checking assumptions required for model fitting and hypothesis testing, and handling missing values and making transformations of variables as needed. EDA encompasses IDA.

<img src='./diagrams/legos.jpeg'>

# Exploratory Data Analysis is...
- Detective work  
- Will inform what is feasible  
- Cornerstone of most analytics projects  
- Might result in the toughest challenges you face in analytics  
- Will likely take the bulk of your time  
- Likely need to continually revisit during a project  
- Messy in the real world

# Exploratory Data Analysis Could Include...
- Consolidating/aggregating/merging data  
- Extracting from non-tabular datasets
- Understanding distributions, shape, and characteristics of your data  
- Casting data to its proper type (e.g., string to date)  
- Determining where there are gaps (so you can get more data)  
- Intermediate data objects to support specific analysis  
- Schemas to address missing data and/or outliers  
- Lots and lots and lots of plots  
- Creativity  

# And You'll Probably Find...
- Numbers stored as strings, strings as numbers, …  
- Lack of metadata on ordinal data  
- Aggregations required  
- Reconstructing what history looked like at time of events (record updates)  
- Significant changes in business processes that create structural shifts  
- Data is scattered across hundreds of Excel files  
- You need to engineer supplemental data  
- NULL means different things at different times. 

# Goals

The objectives of EDA are to:

- Suggest hypotheses about the causes of observed phenomena  
- Assess assumptions on which statistical inference will be based  
- Support the selection of appropriate statistical tools and techniques  
- Provide a basis for further data collection through surveys or experiments

# A minute on Tidy Data
http://vita.had.co.nz/papers/tidy-data.pdf

> A huge amount of effort is spent cleaning data to get it ready for analysis, but there has been little research on how to make data cleaning as easy and effective as possible. This paper tackles a small, but important, component of data cleaning: data tidying. **Tidy datasets are easy to manipulate, model and visualise, and have a specific structure: each variable is a column, each observation is a row, and each type of observational unit is a table.** This framework makes it easy to tidy messy datasets because only a small set of tools are needed to deal with a wide range of un-tidy datasets. This structure also makes it easier to develop tidy tools for data analysis, tools that both input and output tidy datasets. The advantages of a consistent data structure and matching tools are demonstrated with a case study free from mundane data manipulation chores.

<img src='./diagrams/tidy-data.png'>

# Example: Tidying 
Data: Football Salaries  
Accessed Source: https://github.com/rfordatascience/tidytuesday/tree/master/data/2018/2018-04-09  
Original source: https://www.spotrac.com/rankings/

This example is to highlight some ways you may want to explore numeric data, e.g., looking at the distributions, getting summary statistics, and looking at how the data varies by dimensional attributes. 

Salaries are in dollars and each observation is going to be an individual player's salary.

If you are new to American Football:

<img src='./diagrams/football-positions.png'>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

nfl = pd.read_excel('https://github.com/msaricaumbc/DS_data/blob/master/ds602/nfl_salary.xlsx?raw=true')
print(f'Rows: {nfl.shape[0]:,} | Columns: {nfl.shape[1]:,}')
nfl.head()

### This violates the tidy principles (similar to the Census data and the WHO data).
Let's melt it, so each observation is a row (an individual player's salary for a given year).

In [ ]:
nflLong = nfl.melt(id_vars='year', var_name='position', value_name='salary')
nflLong.info()

In [ ]:
nflLong.head()

__Drop the NULL records. Looks like some positions didn't have an equal number of observations entered for each year__

In [ ]:
nflLong = nflLong.dropna()
nflLong.info()

__We have: two dimensions: time (year), position, and a numerical value.__
For numerical data, the standard summary is the 5-number summary:
- Minimum.  
- 1st quartile. 
- Median. 
- 3rd quartile.  
- Maximum.  

And the standard moments:  
- Mean.  
- Variance or standard deviation.  

In [ ]:
# /1000 to scale in ths.
(nflLong['salary'].describe()/1000).round(2)

### Aggregate Summaries

In [ ]:
nflLong.groupby('year')['salary'].count().sort_index()

### Summaries with pivot tables

>A pivot table is a table of grouped values that aggregates the individual items of a more extensive table (such as from a database, spreadsheet, or business intelligence program) within one or more discrete categories. This summary might include sums, averages, or other statistics, which the pivot table groups together using a chosen aggregation function applied to the grouped values.
<br><br>Pivot tables are a technique in data processing. They arrange and rearrange (or "pivot") statistics in order to draw attention to useful information. This leads to finding figures and facts quickly making them integral to data analysis. This ultimately leads to helping businesses or individuals make educated decisions.
<br><br>Although pivot table is a generic term, Microsoft trademarked PivotTable in the United States in 1994 (canceled in 2020)
[Wikipedia](https://en.wikipedia.org/wiki/Pivot_table)

In [ ]:
meanNflPivot = (
    (nflLong.pivot_table(index='position', columns='year', values='salary', aggfunc='median')/1000000)
    .round(1)
)

meanNflPivot

#### Adding a splash of color with heat maps
[Heat maps](https://en.wikipedia.org/wiki/Heat_map)

In [ ]:
import seaborn as sns

sns.heatmap(meanNflPivot)
plt.xlabel('')
plt.ylabel('')
plt.title('Median Salary by Position and Year')
plt.show()

### Trends in Salary

In [ ]:
yrAvg = nflLong.groupby('year')['salary'].mean()
yrErr = nflLong.groupby('year')['salary'].std()

fig, ax = plt.subplots()
yrAvg.plot.bar(yerr=yrErr, ax=ax, capsize=4, rot=0)
plt.title('Mean Salary Per Year (w/ Error Bars)')
plt.xlabel('')
plt.show()

__Looks like it is increasing over time, that would be important to know.__

### Distributions

In [ ]:
nflLong['salary'].hist(bins=100)
plt.title('Salary Distribution\nAll years pooled', loc='right')
plt.show()

## Log Transforms will compress the variance
Also useful if performing regression, log-transforms will help with some techical issues regarding correlations to residuals, and will generally model out better. For money ($$) data, the coefficients will be elasticities, which are nice for presenting to business users.

In [ ]:
np.log(nflLong['salary']).hist(bins=100)
plt.title('Salary Distribution\nAll years pooled', loc='right')
plt.show()

In [ ]:
nflLong['salaryBin'] = (nflLong['salary'] / 1000000).astype(int)
nflLong.head()

In [ ]:
nflSalaryPivot = nflLong.pivot_table(index='salaryBin', columns='year', values='salary', aggfunc='count')
nflSalaryPivot

In [ ]:
nflSalaryPivot.plot(alpha=0.5)
plt.legend(title='')
plt.xlabel('Salary ($millions)')
plt.show()

#### Boxplots for looking at the variance between groups

In [ ]:
# nflLong.boxplot(column='salary', by='position', vert=False, showfliers=True)
nflLong.boxplot(column='salary', by='position', vert=False, showfliers=False)
plt.grid(False)
plt.title('')
plt.xlabel('Salary')
plt.show()

__What can we say about the salaries by position?__

# Numerical Comparisons
Data: iris

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/iris.csv')
df.info()

#### Data summaries by groups

In [ ]:
df.groupby('species').describe().T


### Picture is worth 1,000 words (or 96 cells)

In [ ]:
df

In [ ]:
subs = df.boxplot(by='species')

fig = subs[0][0].get_figure()
fig.suptitle('Grouped by Species')

rotation = 45
for s in subs:
    x1, x2 = s
    
    for m in x1.get_xticklabels():
        m.set_rotation(rotation)
        
    for m in x2.get_xticklabels():
        m.set_rotation(rotation)
        
plt.show()

### Scatterplots and Histograms

In [ ]:
pd.plotting.scatter_matrix(df, figsize=(6,6))
plt.show()

__Scatterplot matrices are a valuable tool, but lose their utility as the number of features grow.__

### Parallel Coordinates
Sometimes scatter plot matrices get too crowded and it's hard to see co-movement between different features.

From Wikipedia:
> To show a set of points in an n-dimensional space, a backdrop is drawn consisting of n parallel lines, typically vertical and equally spaced. A point in n-dimensional space is represented as a polyline with vertices on the parallel axes; the position of the vertex on the i-th axis corresponds to the i-th coordinate of the point.   
This visualization is closely related to time series visualization, except that it is applied to data where the axes do not correspond to points in time, and therefore do not have a natural order. Therefore, different axis arrangements may be of interest.

**Warning: Make sure your data is on the same scale, or this won't be as useful.**

In [ ]:
pd.plotting.parallel_coordinates(df, class_column='species')
plt.show()

### Andrews Curve
From Wikipedia:

> In data visualization, an Andrews plot or Andrews curve is a way to visualize structure in high-dimensional data. It is basically a rolled-down, non-integer version of the Kent–Kiviat radar m chart, or a smoothed version of a parallel coordinate plot. It is named after the statistician David F. Andrews.

In [ ]:
pd.plotting.andrews_curves(df, 'species')
plt.show()

### Reducing the number of dimensions with Principal Component Analysis (PCA)
[Principal Component Analysis](https://en.wikipedia.org/wiki/Principal_component_analysis)

An unsupervised technique to reduce the feature space into **N** orthogonal vectors that eventually capture all the variation in the feature space. Typically used for reducing dimensionality and/or visualization.

It is a handy trick for reducing the number of features from **N** to **2** in order to plot on a scatterplot.
- If the 2 components explain a large amount of the variance this can be very helpful.  
- If the 2 components don't explain a significant amount of the variance, it can be misleading.  
- PCA is very scale sensitive, so be careful.  
- We'll discuss this is more detail later.  

In [ ]:
#a quick reminder of the data
df.head(2)

In [ ]:
sns.relplot(x='sepal_length', y='sepal_width', hue='species', data=df)
plt.show()

In [ ]:
df.iloc[:, :4]

In [ ]:
from sklearn.decomposition import PCA
import seaborn as sns

pca = PCA(n_components=2)
dfPca = pd.DataFrame(pca.fit_transform(df.iloc[:, :4]), columns=['pc1','pc2'])
dfPca

In [ ]:
dfM = df.join(dfPca)
dfM.head()

In [ ]:
sns.relplot(x='pc1', y='pc2', hue='species', data=dfM)
plt.show()

# Joining Data
In organization, most of the time you'll need to assemble your data from multiple sources. There may be multiple major and intermediate steps to get to the point where you have your training data.

<img src='./diagrams/join-pipeline.png'>

### Sometimes how to configure the join might be a little tricky, but here's a guide:

<img src='./diagrams/sql-join2.png'>

### Example: We want to determine relationship of urban population rates to taxes.
- File 1: urban population. 
- File 2: taxes.

In [ ]:
import pandas as pd
import numpy as np

population = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/share-of-population-urban.csv')
taxes = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/taxes-on-incomes-of-individuals-and-corporations-gdp.csv')

population.info()
print('\n-------------------------------------\n')
taxes.info()

### Left Join

In [ ]:
leftJoin = pd.merge(population, taxes, how='left', on=['Entity','Code','Year'])
leftJoin.shape

In [ ]:
leftJoin.info()

### Inner Join

In [ ]:
innerJoin = pd.merge(population, taxes, how='inner', on=['Entity','Code','Year'])
innerJoin.shape

In [ ]:
innerJoin.head()

In [ ]:
innerJoin[['Urban_Population','Tax_Percent_GDP']].corr()

<img src='./diagrams/what.png'>

__Population is a string! Need to cast it to a numeric value.__

In [ ]:
innerJoin.info()

In [ ]:
innerJoin['Population'] = pd.to_numeric(innerJoin['Urban_Population'], errors='coerce')
innerJoin.info()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

innerJoin['Population'].hist()
plt.title('Urban Population %')
plt.show()

In [ ]:
innerJoin[['Population','Tax_Percent_GDP']].corr()

In [ ]:
innerJoin[['Population','Tax_Percent_GDP']].plot.scatter(x='Population', y='Tax_Percent_GDP', alpha=.1)
plt.xlabel('% Urban Population')
plt.ylabel('Tax (% GDP)')
plt.show()

# Concatenating Data
- Sometimes do to data collection convenience or file sizes, you may need to deal with a lot of files  
- Sometimes the structure will be identical so you can effectively concatenate them together  
- In order to loop through them you’ll likely need to use some of the base Python data structures (e.g., list, dictionary), loops, and/or the base libraries to make interacting with the file system more convenient  


<img src='./diagrams/excel-pipeline.png'>

### Example data: https://www.ssa.gov/OACT/babynames/limits.html

In [ ]:
# !pip install zipfile

import requests
from zipfile import ZipFile

def download_file(file_name, url):
    # Use requests Session with comprehensive browser headers
    session = requests.Session()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': 'https://www.ssa.gov/',
        'Origin': 'https://www.ssa.gov',
        'Connection': 'keep-alive',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
    }
    session.headers.update(headers)
    
    # First, try to get the main page to establish a session
    try:
        session.get('https://www.ssa.gov/OACT/babynames/state/', timeout=10)
    except:
        pass  # Continue even if this fails
    
    # Now download the file
    response = session.get(url, headers=headers, timeout=30, allow_redirects=True)
    response.raise_for_status()  # Raise an error for bad status codes
    
    # Ensure the directory exists
    import os
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'Successfully downloaded {file_name}')
        
def unzip(file_name, path='./'):
    with ZipFile(file_name, 'r') as zip: 
        zip.printdir() 
        print('Extracting all the files now...') 
        zip.extractall(path = path) 
        print('Done!')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

data_url = 'https://www.ssa.gov/OACT/babynames/state/namesbystate.zip'
filename='data/namesbystate.zip'
fldr = 'data/namesbystate'

if os.path.exists(fldr) == False:
    print("data folder doesn't exist... creating & downloading data files")
    os.makedirs(fldr)
    download_file(filename, data_url)
    unzip(filename, fldr)

fldrFiles = os.listdir(fldr)

print(f'File count: {len(fldrFiles)}')

#### Analyze the types of files

In [ ]:
fldrExt = {}
for f in fldrFiles:
    extention = f.split('.')[-1].lower()
    fldrExt[extention] = fldrExt.get(extention, 0) + 1
    
fldrExt

#### Loop through and import

In [ ]:
filesImport = [x for x in fldrFiles if x.lower().endswith('.txt')]

dfHolder = {}
for f in filesImport:
    dfHolder[f.split('.')[0]] = pd.read_csv(os.path.join(fldr, f),
                                            header=None,
                                            names=['state','gender','year','name','count'])
    
print(f'Number of files: {len(dfHolder)}')

#### Union the files

In [ ]:
df = pd.concat(dfHolder, axis=0, ignore_index=True)

dfx, dfy = df.shape
print(f'Rows: {dfx:,}\nColumns: {dfy:,}')

In [ ]:
df.head()

In [ ]:
df.info()

### Analysis Ready

In [ ]:
df.groupby('year')['count'].sum().plot()
plt.title('Number of Babies over time', loc='right')
plt.xlabel('')
plt.show()

#### Most popular names

In [ ]:
# top 5 names for each gender over the past 5 years:
(
    df.query('year >= 2016')
    .groupby(['gender','name'])['count']
    .sum()
    .groupby('gender')
    .nlargest(5)
)

### Testing the influence of celebrities on naming trends
Can we associate naming trends with celebrity popularity?
<img src='./diagrams/elvis.jpg'>
Image Source: Wikipedia

#### Albums:
- Studio albums  
- Elvis Presley (1956)  
- Elvis (1956)  
- Elvis' Christmas Album (1957)  
- Elvis is Back! (1960)  
- His Hand in Mine (1960)  
- Something for Everybody (1961)  
- Pot Luck (1962)  
- Elvis for Everyone! (1965)  
- How Great Thou Art (1967)  
- From Elvis in Memphis (1969)  
- From Memphis to Vegas / From Vegas to Memphis (1969)  
- That's the Way It Is (1970)  
- Elvis Country (I'm 10,000 Years Old) (1971)  
- Love Letters from Elvis (1971)  
- Elvis sings The Wonderful World of Christmas (1971)  
- Elvis Now (1972)  
- He Touched Me (1972)  
- Elvis (1973) (The "Fool" Album)  
- Raised on Rock / For Ol' Times Sake (1973)  
- Good Times (1974)  
- Promised Land (1975)  
- Today (1975)  
- From Elvis Presley Boulevard, Memphis, Tennessee (1976)  
- Moody Blue (1977)  

In [ ]:
def check_name_popularity_over_years(name):
    print(sum(df.name == name))
    df.query('name==@name').groupby('year')['count'].sum().plot()
    plt.title(f'Trend of Babies Named "{name}"', loc='center')
    plt.xlabel('')
    plt.ylabel('')
    plt.show()

In [ ]:
sum(df.name == 'Elvis')

In [ ]:
check_name_popularity_over_years('Elvis')

In [ ]:
check_name_popularity_over_years('Ariana')

In [ ]:
check_name_popularity_over_years('Elon')

In [ ]:
check_name_popularity_over_years('Alexa')
# why Alexa name dropped ??

In [ ]:
# check_name_popularity_over_years('Taylor')

# Resources
[Edward Tufte](https://www.edwardtufte.com/tufte/)
<br>[Stephen Few](https://www.perceptualedge.com)
<br>[Save The Pies for Dessert](https://www.perceptualedge.com/articles/visual_business_intelligence/save_the_pies_for_dessert.pdf)
<br>[Tableau Public Gallery](https://public.tableau.com/en-us/gallery/?tab=viz-of-the-day&type=viz-of-the-day)
<br>[Matplotlib](https://matplotlib.org)
<br>[seaborn](https://seaborn.pydata.org)
<br>[JunkCharts - examples of what not to do](https://junkcharts.typepad.com)


# Some refreshers

https://github.com/msaricaumbc/DS601/blob/main/Week03/week03_2.ipynb

https://github.com/msaricaumbc/DS601/blob/main/Week04/week04%20-%20data%20transpormation%202.ipynb

https://github.com/msaricaumbc/DS601/blob/main/Week07/week7%20relational%20data.ipynb